## Tensor

In [1]:
import torch
import numpy as np

In [2]:
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torchvision
from torchvision import transforms
import time

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:

TRAIN_PATH = "./asl_alphabet_train"

TRANSFORM = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    #20transforms.Resize(64),
    transforms.ToTensor()
    ])
train_set = ImageFolder(root = TRAIN_PATH, transform = TRANSFORM)
train_set_loader = DataLoader(train_set, batch_size=10, shuffle=True,  num_workers=4)


In [5]:
for i in range(50):
    img, label = train_set[-i]
    print(label)

0
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25


classes

In [6]:
classes = ('A', 'B', 'C', 'D',
           'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U'
          'V', 'W', 'X', 'Y', 'Z')


n_training_samples = 2000



In [7]:
from torch.autograd import Variable
import torch.nn.functional as F

class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(1, 26, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(26 * 100 * 100, 200)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(200, 26)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)

        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 26 * 100 *100)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [8]:
model = SimpleCNN()

#x = torch.rand(10,1,32,32)
#y = model(x)
#y.size()

In [33]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [34]:
loss = torch.nn.CrossEntropyLoss()

a1 = torch.Tensor(np.eye(4)).float()
a2 = torch.zeros(4).long()
loss(a1,a2)

tensor(1.4937)

In [ ]:
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = train_set_loader
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            labels=labels.long()
            #onehotlabels = onehotlabels.long()
            
                    
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)#.long()
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data[0]
            total_train_loss += loss_size.data[0]
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for data in val_loader:
            inputs, labels = data
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels.long())
            total_val_loss += val_loss_size.data[0]
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

trainNet(model, 40, 2, .001)    

===== HYPERPARAMETERS =====
batch_size= 40
epochs= 2
learning_rate= 0.001


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:47: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Construct a tensor directly from data

Construct a matrix filled zeros

In [ ]:
x = torch.zeros(6, 4, dtype=torch.float)
print(x)

Construct a randomly initialized matrix

In [ ]:
x = torch.rand(, 3)
print(x)

Basic operation: addition

In [ ]:
y = torch.rand(5, 3)
print(x)
print(y)
print(x + y)

## Neural Net

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

Below defines our neural net structure (important)

In [ ]:
# Our neural net structure:
# input_tensor -> linear_layer_1 -> activation_func_1 -> linear_layer_2 -> activation_func_2 -> linear_layer_3 -> output_tensor

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.L1 = nn.Linear(3*32*32, 120) # 3*32*32 is the size of a picture's input matrix
        self.F1 = F.relu
        self.L2 = nn.Linear(120, 84)
        self.F2 = F.relu
        self.L3 = nn.Linear(84, 10)

    def forward(self, x):
        x = x.view(-1, 3*32*32) # sequentialize a picture's input matrix to a tensor
        L1_out = self.L1(x)
        F1_out = self.F1(L1_out)
        L2_out = self.L2(F1_out)
        F2_out = self.F2(L2_out)
        L3_out = self.L3(F2_out)
        y = L3_out
        return y
    
net = Net()

## Load a sample dataset

Load CIFAR-10 dataset (not important)

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

train_dataiter = iter(trainloader)
test_dataiter = iter(testloader)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# get some random training images
images, labels = train_dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

## Train our model

Loss function and optimization algorithm (important but skippable)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Start training

In [ ]:
for epoch in range(2): # loop over the dataset multiple times, use larger number if time allows

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

## Result

Ground truth

In [ ]:
images, labels = test_dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

Our prediction

In [ ]:
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

## Resources

https://pytorch.org/docs/stable/nn.html

## Things to try
1. Find out the accuracy of our model
2. Add more layers to our neural net and see if it improves the performance
3. Check the above link to learn about more advanced layers than nn.Linear